In [1]:
import modelWin_env as modelWin
import discrete_MDP_transition_based_rewards as MDP
import generate_dataset
import numpy as np

In [2]:
s0 = 0
env = MDP.discrete_MDP_transition_based_rewards(modelWin.state_transition_matrix,
                                                modelWin.transition_based_rewards)
env.reset(s0)

In [3]:
n = 10000; horizon = 5
D = generate_dataset.generate_dataset(env, s0, horizon, n,
                                      modelWin.behavior_action_matrix,
                                      modelWin.evaluation_policy_matrix)
D.shape
# columns of D: s, a, r, pi_b, pi_e, rho_t

(10000, 5, 6)

In [4]:
Q0, V0 = env.get_true_Q_and_V_functions(modelWin.evaluation_policy_matrix, 1, horizon)
V0[0,:]

array([0.276, 0.184, 0.184])

In [5]:
def IPS(D):
    horizon = D.shape[1]
    return np.mean(D[:, horizon-1, 5] * np.sum(D[:, :, 2], axis=1))

In [6]:
def stepIPS(D):
    return np.mean(np.sum(D[:, :, 5] * D[:, :, 2], axis=1) )

In [7]:
def WIPS(D):
    horizon = D.shape[1]
    w_H = np.mean(D[:, horizon-1, 5])
    return np.mean(D[:, horizon-1, 5] / w_H * np.sum(D[:, :, 2], axis=1))

In [8]:
def stepWIPS(D):
    n = D.shape[0]
    horizon = D.shape[1]
    w_t = np.mean(D[:, :, 5], axis=0)
    return np.mean(np.sum(D[:, :, 5] / (np.ones((n,1)) * w_t.T) * D[:, :, 2], axis=1))

In [36]:
def DR(D):
    n = D.shape[0]
    horizon = D.shape[1]
    D_star = np.zeros((n, horizon))

    t = horizon-1
    D_star[:, t] = D[:, t, 5] * (D[:, t, 2] - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    for t in np.arange(0, horizon-1)[::-1]:
        D_star[:, t] = D[:, t, 5] * (D[:, t, 2] + V_hat[t+1, D[:, t+1, 0].astype(int)]
                                      - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    return np.mean(V_hat[t, D[:, t, 1].astype(int)] + np.sum(D_star, axis=1))

In [38]:
def WDR(D):
    n = D.shape[0]
    horizon = D.shape[1]
    D_star = np.zeros((n, horizon))
    w_t = np.mean(D[:, :, 5], axis=0)
    
    t = horizon-1
    D_star[:, t] = D[:, t, 5] / w_t[t] * (D[:, t, 2] - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    for t in np.arange(0, horizon-1)[::-1]:
        D_star[:, t] = D[:, t, 5] / w_t[t] * (D[:, t, 2] + V_hat[t+1, D[:, t+1, 0].astype(int)]
                                      - Q_hat[t, D[:, t, 0].astype(int), D[:, t, 1].astype(int)])
    return np.mean(V_hat[t, D[:, t, 1].astype(int)] + np.sum(D_star, axis=1))

In [39]:
print(IPS(D))
print(stepIPS(D))
print(WIPS(D))
print(stepWIPS(D))
print(DR(D))
print(WDR(D))

0.2535354956054595
0.2878522817724615
0.25345980619991576
0.28646929528377246
0.26104379853101006
0.26109588803976397
